In [1]:
#svm
import menpo.io as mio
path_to_svm_database = '/Programing/GR/Code/CK+/aam-images/**/**/**/*'

def process(image, crop_proportion=0.2, max_diagonal=400):
    if image.n_channels == 3:
        image = image.as_greyscale()
    image = image.crop_to_landmarks_proportion(crop_proportion)
    d = image.diagonal()
    if d > max_diagonal:
        image = image.rescale(float(max_diagonal) / d)
    return image

training_images = mio.import_images(path_to_svm_database, verbose=True)
training_images = training_images.map(process)

Found 1186 assets, index the returned LazyList to import.


In [29]:
import os
class ChangeVector:
    def __init__(self, facs = [], landmarkChange = []):
        self.landmarkChange = landmarkChange
        self.facs = facs

path_to_facs = '/Programing/GR/Code/CK+/FACS/'
path_to_emotions = '/Programing/GR/Code/CK+/Emotion/'
count = 0;
svm_training_data = []
while(count < 1186):
    file_path = str(training_images[count].path).split("\\")
    
    facs_path = path_to_facs + file_path[6] + '/' + file_path[7]
    facs_path = facs_path + '/' + os.listdir(facs_path)[0]
    fi = open(facs_path, 'r')
    facs = []
    for line in fi: # read rest of lines
        for x in line.split():
            facs.append(int(float(x)))
    #print(array)
    fi.close()
    
    landmarkChange = []
    landmark_neutral = training_images[count].landmarks['PTS'].lms.points
    landmark_perk = training_images[count + 1].landmarks['PTS'].lms.points
    for i in range(0,68):
        landmarkChange.append([landmark_perk[i][0] - landmark_neutral[i][0], landmark_perk[i][1] - landmark_neutral[i][1]])
    
    svm_training_data.append(ChangeVector(facs, landmarkChange))   
    count = count + 2

In [12]:
print(svm_training_data[400].facs)

[12.0, 3.0]


In [46]:
from sklearn import svm
models = []
facs = []
#create facs array
for data in svm_training_data:
    for facs_code in data.facs:
        if(int(facs_code) not in facs):
            facs.append(facs_code)

#create model for each action unit in facs[]
for au in facs:
    x_training = []
    y_label = []
    #create label array
    for data in svm_training_data:
        if(au in data.facs):
            y_label.append(1)
        else:
            y_label.append(0)
        
        #create training data: 1x136 array
        tmp = []
        for vector in data.landmarkChange:
            tmp.append(vector[0])
            tmp.append(vector[1])
        x_training.append(tmp)
    clf = svm.SVC()
    clf.fit(x_training, y_label)
    models.append(clf)


In [41]:
print(models[0])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [51]:
data = svm_training_data[30]
for au in data.facs:
    model_index = facs.index(au)
    tmp = []
    for vector in data.landmarkChange:
        tmp.append(vector[0])
        tmp.append(vector[1])
    print(au)
    print(models[model_index].predict([tmp]))


4
[1]
4
[1]
7
[1]
3
[1]
14
[1]
2
[1]
24
[1]
3
[1]
17
[1]
5
[1]
[0]
